<a href="https://colab.research.google.com/github/passah2o/pygeoflood/blob/master/examples/fim_workflow_indiv_steps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


### Install pygeoflood
See README.md installation instructions.

On Google Colab, run the following command to install pygeoflood:
```bash
%pip install git+https://github.com/passah2o/pygeoflood
```

### Import libraries


In [ ]:
from pathlib import Path
from pygeoflood import PyGeoFlood

### Instantiate PyGeoFlood model class


In [ ]:
pgf = PyGeoFlood()

### Set paths to input datasets with our model instance


In [ ]:
pgf.dem_path = "data/OC1mTest.tif"
pgf.flowline_path = "data/Flowline.shp"
pgf.catchment_path = "data/Catchment.shp"

# # we are not using these datasets for this example
# pgf.streamflow_forecast_path = "data/prod_nwm.t00z.analysis_assim.channel_rt.tm00.conus.nc"
# pgf.custom_flowline_path = Path(data_dir, "NHDPlus_H_1209_HU4_GDB.gdb")

### Download input datasets if necessary

In [ ]:
# make data directory if it doesn't exist
if not Path("data").is_dir():
    Path("data").mkdir()
# download input datasets if they doesn't exist
# DEM from USGS 3DEP
if not pgf.dem_path.is_file():
    !curl https://utexas.box.com/shared/static/id0in7w8yn38o59sexa50aqunsr7mzk0.tif -Lso data/OC1mTest.tif
# flowlines and catchments from NFIE (NHD MR) dataset
# https://www.arcgis.com/home/webmap/viewer.html?webmap=9766a82973b34f18b43dafa20c5ef535
if not pgf.flowline_path.is_file():
    !curl -sL https://utexas.box.com/shared/static/s1vlnymonbidnhul52oja7s83ky1212j.tgz | tar -xz -C data
if not pgf.catchment_path.is_file():
    !curl -sL https://utexas.box.com/shared/static/jzawwvmb2tjl4yqtnitgvfjcc4ho4tg5.tgz | tar -xz -C data

### Run GeoFlood operations


In [9]:
# apply nonlinear filter to DEM
pgf.apply_nonlinear_filter()

Using parameters:
    custom_path = None
    method = PeronaMalik2
    smoothing_quantile = 0.9
    time_increment = 0.1
    n_iter = 50
    sigma_squared = 0.05
    method_name = apply_nonlinear_filter
Computing slope of raw DTM
DEM slope array shape: (3378, 4750)
Computing lambda = q-q-based nonlinear filtering threshold
DEM smoothing Quantile: 0.9
Edge Threshold Value: 0.34356024861335754
Filtered DEM written to data/OC1mTest_filtered.tif

apply_nonlinear_filter completed in 9.2257 seconds


In [6]:
# calculate slope
pgf.calculate_slope()

 slope statistics
 min angle: 0.0
 max angle: 54.863846043721324
 mean slope: 0.13960592
 stdev slope: 0.15432632
Slope raster written to data/OC1mTest_slope.tif

calculate_slope completed in 1.1411 seconds


In [7]:
# calculate curvature
pgf.calculate_curvature()

 using geometric curvature
 curvature statistics
 non-nan curvature cell number: 16045500
 non-nan finite curvature cell number:  mean:  -0.00031314598
 standard deviation:  0.112147875
Curvature raster written to data/OC1mTest_curvature.tif

calculate_curvature completed in 1.1721 seconds


In [8]:
# fill depressions on filtered DEM
pgf.fill_depressions()

Decompressing WhiteboxTools_darwin_amd64.zip ...
WhiteboxTools package directory: /Users/markwang/micromamba/envs/wip-geoflood-env/lib/python3.11/site-packages/whitebox
Filled DEM written to data/OC1mTest_filled.tif

fill_depressions completed in 44.8331 seconds


In [9]:
# calculate MFD flow accumulation
pgf.calculate_mfd_flow_accumulation()

MFD flow accumulation raster written to data/OC1mTest_mfd_fac.tif

calculate_mfd_flow_accumulation completed in 4.6939 seconds


In [10]:
# calculate D8 flow directions
pgf.calculate_d8_flow_direction()

D8 flow direction raster written to data/OC1mTest_d8_fdr.tif

calculate_d8_flow_direction completed in 1.6886 seconds


In [11]:
# calculate outlets
pgf.find_outlets()

Outlets raster written to data/OC1mTest_outlets.tif

find_outlets completed in 0.2165 seconds


In [12]:
# delineate basins
pgf.delineate_basins()

Basins raster written to data/OC1mTest_basins.tif

delineate_basins completed in 0.9526 seconds


In [13]:
# define channel skeletons
pgf.define_skeleton()

Curvature mean:  -0.00031413726
Curvature standard deviation:  0.11232527
Curvature Projection: EPSG:6343
Mean upstream flow:  2617.3586
Combined skeleton written to data/OC1mTest_combined_skeleton.tif

define_skeleton completed in 0.8064 seconds


In [14]:
# geodesic distance (not needed for GeoFlood workflow)
pgf.calculate_geodesic_distance(write_cost_function=True)

Performing fast marching
FMM iteration 1/32: RAM usage: 9.91/16.00 GB (61.9%)
FMM iteration 2/32: RAM usage: 9.84/16.00 GB (61.5%)
FMM iteration 3/32: RAM usage: 9.81/16.00 GB (61.3%)
FMM iteration 4/32: RAM usage: 9.81/16.00 GB (61.3%)
FMM iteration 5/32: RAM usage: 9.81/16.00 GB (61.3%)
FMM iteration 6/32: RAM usage: 9.81/16.00 GB (61.3%)
FMM iteration 7/32: RAM usage: 9.80/16.00 GB (61.3%)
FMM iteration 8/32: RAM usage: 9.80/16.00 GB (61.3%)
FMM iteration 9/32: RAM usage: 9.80/16.00 GB (61.3%)
FMM iteration 10/32: RAM usage: 9.91/16.00 GB (61.9%)
FMM iteration 11/32: RAM usage: 9.92/16.00 GB (62.0%)
FMM iteration 12/32: RAM usage: 9.92/16.00 GB (62.0%)
FMM iteration 13/32: RAM usage: 9.92/16.00 GB (62.0%)
FMM iteration 14/32: RAM usage: 9.92/16.00 GB (62.0%)
FMM iteration 15/32: RAM usage: 9.86/16.00 GB (61.6%)
FMM iteration 16/32: RAM usage: 9.85/16.00 GB (61.6%)
FMM iteration 17/32: RAM usage: 9.86/16.00 GB (61.6%)
FMM iteration 18/32: RAM usage: 9.86/16.00 GB (61.6%)
FMM iteratio

In [15]:
# get channel heads (not needed for GeoFlood workflow)
pgf.identify_channel_heads()

Locating skeleton end points
Counting the number of elements of each connected component
Skeleton region size threshold: 258.7368421052632
Continuing to locate skeleton endpoints
Channel heads shapefile written to data/OC1mTest_channel_heads.shp

identify_channel_heads completed in 2.9383 seconds


In [16]:
# get flowline endpoints
pgf.find_endpoints()

Endpoints csv written to data/OC1mTest_endpoints.csv

find_endpoints completed in 0.0244 seconds


In [17]:
# get binary HAND array
pgf.calculate_binary_hand()

Binary HAND raster written to data/OC1mTest_binary_hand.tif

calculate_binary_hand completed in 4.0729 seconds


In [18]:
# # get custom flowline from NHD HR dataset (optional)
# pgf.rasterize_custom_flowline(layer="NHDFlowline")

In [19]:
# extract channel network
pgf.extract_channel_network()

Cost function weights:
curvature          0.2520 (mean flow accumulation)
mfd_fac            1.0000
binary_hand        0.7500
custom_flowline    0.0000

Cost min: 0.5028837585834142
Cost max: inf
cost shape: (3378, 4750)
Creating path 1/1: RAM usage: 9.89/16.00 GB (61.8%)
Number of endpoints: 1
Channel network raster written to data/OC1mTest_channel_network.tif
Channel network vector written to data/OC1mTest_channel_network.shp

extract_channel_network completed in 1.8860 seconds


In [20]:
# calculate HAND
pgf.calculate_hand()

HAND raster written to data/OC1mTest_HAND.tif

calculate_hand completed in 4.7869 seconds


In [21]:
# segment flowline
pgf.segment_channel_network()

Segmented channel network written to data/OC1mTest_segmented_channel_network.shp

segment_channel_network completed in 0.7895 seconds


In [22]:
# delineate channel network segment catchments
pgf.delineate_segment_catchments()

Segment catchments written to data/OC1mTest_segment_catchments.tif

delineate_segment_catchments completed in 1.4397 seconds


In [23]:
# calculate synthetic rating curves table
pgf.calculate_src()

River attributes written to data/OC1mTest_river_attributes.csv
Synthetic rating curves written to data/OC1mTest_src.csv

calculate_src completed in 2.8342 seconds


In [24]:
# calculate flood stage from synthetic rating curves
# pgf.streamflow_forecast_path must be set or custom_Q must be provided
pgf.calculate_flood_stage(custom_Q=500)

Applying custom streamflow to each segment: 500 cms
Flood stages written to data/OC1mTest_flood_stage.csv

calculate_flood_stage completed in 0.0069 seconds


In [1]:
# create flood depth grid (FIM)
pgf.inundate()

Flood inundation raster written to data/OC1mTest_fim.tif

inundate completed in 1.1381 seconds


In [2]:
# save output data file paths we can load our PyGeoFlood instance later with
# pgf = PyGeoFlood.from_paths("data/paths.txt")
pgf.to_paths("data/paths.txt")

Paths written to data/paths.txt
